# Importer les données

In [2086]:
using CSV, DataFrames, Statistics, Dates, Gadfly,LinearAlgebra, Distributions
import StatsBase, Random

In [2087]:
essence = CSV.read("data/essence.csv", DataFrame)
recolte = CSV.read("data/recolte.csv", DataFrame)
permis = CSV.read("data/permis.csv", DataFrame)
test = CSV.read("data/test.csv", DataFrame)
meteo = CSV.read("data/meteo.csv", DataFrame)

,Date,Mean Temp (°C),Total Rain (mm),Total Snow (cm),Snow on Grnd (cm)
,Date,Float64?,Float64?,Float64?,Int64?
1,1999-01-01,-20.5,0.0,1.0,2
2,1999-01-02,-19.2,0.0,0.0,2
3,1999-01-03,-8.5,1.2,11.4,7
4,1999-01-04,-9.4,0.0,0.0,8
5,1999-01-05,-14.3,0.0,0.0,8
6,1999-01-06,-7.1,0.0,3.1,8
7,1999-01-07,-11.2,0.0,0.8,10
8,1999-01-08,-12.8,0.0,5.2,8
9,1999-01-09,-9.9,0.0,13.8,17


# 1. Analyse exploratoire

# 2. Pre processing

## 2.1 Remplacer les données manquantes

On a d'abord remarquer que pour le fichier météo il y avait présence de donnée manquante, on a aussi remarquer que les données était indexé en fonction de la date ( l'année, le mois et le jour ). Pour pallier à cela , on a decider de remplacer ces valeurs manquantes par la moyenne des  valeurs du mois en question ( par exemple si la données manquante était pour le mois 1, on remplace celle-ci par la moyenne des valeurs pour le mois 1).Nous avons opté pour ce choix plutot que de retirer les données afin de garder garder le plus de données possible. De plus la réalisations de la moyenne sur le mois en marticumier nous permet de ne pas trop biaiser les données. Enfin la meme procedure est effectuer pour chaque variables du CSV météo  ou il y'a presence des données manquantes.

On crée un dataframe contenant les mois avec la valeur moyenne de "Snow on Grnd (cm)"

In [2088]:
#Remplacer les valeurs de "snow_on_ground" manquantes et les grouper par années

snowground_par_mois = copy(meteo)
snowground_par_mois = rename(snowground_par_mois, :"Snow on Grnd (cm)" => :SnowGround)
snowground_par_mois[!, "Date"] = [Dates.month(item) for item in snowground_par_mois[!, "Date"]]

snowground_par_mois = dropmissing(snowground_par_mois)
moyenne_par_mois = combine(groupby(snowground_par_mois, :Date), :SnowGround => mean)

,Date,SnowGround_mean
,Int64,Float64
1,1,9.63864
2,2,13.3077
3,3,8.31062
4,4,1.03165
5,5,0.0
6,6,0.0
7,7,0.0
8,8,0.0
9,9,0.0


On remplace les valeurs manquantes de "Snow on Grnd (cm)"  par les valeurs  calculés dans le dataFrame précedemment, puis la valeur dans le dataframe est mise à jour.

In [2089]:

snowground_dropmissing = copy(meteo)
snowground_dropmissing = rename(snowground_dropmissing, :"Snow on Grnd (cm)" => :SnowGround)

n = nrow(snowground_dropmissing)
moyenne_par_mois[!, "SnowGround_mean"][1]
for i=1:n
    if ismissing(snowground_dropmissing[!, "SnowGround"][i])
        snowground_dropmissing[!, "SnowGround"][i] = round(moyenne_par_mois[!, "SnowGround_mean"][Dates.month(snowground_dropmissing[!, "Date"][i])])
    end
end

meteo = snowground_dropmissing


,Date,Mean Temp (°C),Total Rain (mm),Total Snow (cm),SnowGround
,Date,Float64?,Float64?,Float64?,Int64?
1,1999-01-01,-20.5,0.0,1.0,2
2,1999-01-02,-19.2,0.0,0.0,2
3,1999-01-03,-8.5,1.2,11.4,7
4,1999-01-04,-9.4,0.0,0.0,8
5,1999-01-05,-14.3,0.0,0.0,8
6,1999-01-06,-7.1,0.0,3.1,8
7,1999-01-07,-11.2,0.0,0.8,10
8,1999-01-08,-12.8,0.0,5.2,8
9,1999-01-09,-9.9,0.0,13.8,17


On crée un dataframe contenant les mois avec la valeur moyenne de "Total Snow (cm)"

In [2090]:
#Remplacer les valeurs de "snowTotal" manquantes et les grouper par années

snowtotal_par_mois = copy(meteo)
snowtotal_par_mois = rename(snowtotal_par_mois, :"Total Snow (cm)" => :SnowTotal)
snowtotal_par_mois[!, "Date"] = [Dates.month(item) for item in snowtotal_par_mois[!, "Date"]]

snowtotal_par_mois = dropmissing(snowtotal_par_mois)
moyenne_par_mois = combine(groupby(snowtotal_par_mois, :Date), :SnowTotal => mean)


,Date,SnowTotal_mean
,Int64,Float64
1,1,1.62224
2,2,1.6671
3,3,1.10545
4,4,0.38695
5,5,0.000295421
6,6,0.0
7,7,0.0
8,8,0.0
9,9,0.0


On remplace les valeurs manquantes de "Total Snow (cm)"  par les valeurs de la moyenne des mois calculés dans le dataFrame calculé précedemment, puis la valeur de du dataframe est mise à jour.

In [2091]:
snowtotal_dropmissing = copy(meteo)
snowtotal_dropmissing = rename(snowtotal_dropmissing, :"Total Snow (cm)" => :SnowTotal)

n = nrow(snowtotal_dropmissing)
moyenne_par_mois[!, "SnowTotal_mean"][1]
for i=1:n
    if ismissing(snowtotal_dropmissing[!, "SnowTotal"][i])
        snowtotal_dropmissing[!, "SnowTotal"][i] = moyenne_par_mois[!, "SnowTotal_mean"][Dates.month(snowtotal_dropmissing[!, "Date"][i])]
    end
end

meteo = snowtotal_dropmissing

,Date,Mean Temp (°C),Total Rain (mm),SnowTotal,SnowGround
,Date,Float64?,Float64?,Float64?,Int64?
1,1999-01-01,-20.5,0.0,1.0,2
2,1999-01-02,-19.2,0.0,0.0,2
3,1999-01-03,-8.5,1.2,11.4,7
4,1999-01-04,-9.4,0.0,0.0,8
5,1999-01-05,-14.3,0.0,0.0,8
6,1999-01-06,-7.1,0.0,3.1,8
7,1999-01-07,-11.2,0.0,0.8,10
8,1999-01-08,-12.8,0.0,5.2,8
9,1999-01-09,-9.9,0.0,13.8,17


On crée un dataframe contenant les mois avec la valeur moyenne de "Total Rain (mm)"

In [2092]:
#Remplacer les valeurs de "rain" manquantes et les grouper par années

raintotal_par_mois = copy(meteo)
raintotal_par_mois = rename(raintotal_par_mois, :"Total Rain (mm)" => :raintotal)
raintotal_par_mois[!, "Date"] = [Dates.month(item) for item in raintotal_par_mois[!, "Date"]]

raintotal_par_mois = dropmissing(raintotal_par_mois)
moyenne_par_mois = combine(groupby(raintotal_par_mois, :Date), :raintotal => mean)

,Date,raintotal_mean
,Int64,Float64
1,1,0.861561
2,2,0.691248
3,3,1.27187
4,4,2.75099
5,5,2.62777
6,6,3.16391
7,7,2.82548
8,8,3.11047
9,9,2.81366


On remplace les valeurs manquantes de "Total Rain (mm)"  par les valeurs de la moyenne des mois calculés dans le dataFrame  précedemment, puis la valeur de du dataframe est mise à jour.

In [2093]:
raintotal_dropmissing = copy(meteo)
raintotal_dropmissing = rename(raintotal_dropmissing, :"Total Rain (mm)" => :raintotal)

n = nrow(raintotal_dropmissing)
moyenne_par_mois[!, "raintotal_mean"][1]
for i=1:n
    if ismissing(raintotal_dropmissing[!, "raintotal"][i])
        raintotal_dropmissing[!, "raintotal"][i] = moyenne_par_mois[!, "raintotal_mean"][Dates.month(raintotal_dropmissing[!, "Date"][i])]
    end
end
meteo = raintotal_dropmissing

,Date,Mean Temp (°C),raintotal,SnowTotal,SnowGround
,Date,Float64?,Float64?,Float64?,Int64?
1,1999-01-01,-20.5,0.0,1.0,2
2,1999-01-02,-19.2,0.0,0.0,2
3,1999-01-03,-8.5,1.2,11.4,7
4,1999-01-04,-9.4,0.0,0.0,8
5,1999-01-05,-14.3,0.0,0.0,8
6,1999-01-06,-7.1,0.0,3.1,8
7,1999-01-07,-11.2,0.0,0.8,10
8,1999-01-08,-12.8,0.0,5.2,8
9,1999-01-09,-9.9,0.0,13.8,17


On crée un dataframe contenant les mois avec la valeur moyenne de "Mean Temp (°C)"

In [2094]:
#Remplacer les températures manquantes et les grouper par années

meantemp_par_mois = copy(meteo)
meantemp_par_mois = rename(meantemp_par_mois, :"Mean Temp (°C)" => :meantemp)
meantemp_par_mois[!, "Date"] = [Dates.month(item) for item in meantemp_par_mois[!, "Date"]]

meantemp_par_mois = dropmissing(meantemp_par_mois)
moyenne_par_mois = combine(groupby(meantemp_par_mois, :Date), :meantemp => mean)

,Date,meantemp_mean
,Int64,Float64
1,1,-8.69604
2,2,-7.34428
3,3,-1.60073
4,4,6.461
5,5,13.9926
6,6,19.1701
7,7,21.8772
8,8,21.0454
9,9,16.7727


On remplace les valeurs manquantes de "Mean Temp (°C)"  par les valeurs de la moyenne des mois calculés dans le dataFrame calculé précedemment, puis la valeur dans le dataframe est mise à jour.

In [2095]:
meantemp_dropmissing = copy(meteo)
meantemp_dropmissing = rename(meantemp_dropmissing, :"Mean Temp (°C)" => :meantemp)

n = nrow(meantemp_dropmissing)
moyenne_par_mois[!, "meantemp_mean"][1]
for i=1:n
    if ismissing(meantemp_dropmissing[!, "meantemp"][i])
        meantemp_dropmissing[!, "meantemp"][i] = moyenne_par_mois[!, "meantemp_mean"][Dates.month(meantemp_dropmissing[!, "Date"][i])]
    end
end

meteo = meantemp_dropmissing

,Date,meantemp,raintotal,SnowTotal,SnowGround
,Date,Float64?,Float64?,Float64?,Int64?
1,1999-01-01,-20.5,0.0,1.0,2
2,1999-01-02,-19.2,0.0,0.0,2
3,1999-01-03,-8.5,1.2,11.4,7
4,1999-01-04,-9.4,0.0,0.0,8
5,1999-01-05,-14.3,0.0,0.0,8
6,1999-01-06,-7.1,0.0,3.1,8
7,1999-01-07,-11.2,0.0,0.8,10
8,1999-01-08,-12.8,0.0,5.2,8
9,1999-01-09,-9.9,0.0,13.8,17


## 2.2 Création de la matrice X

Afin d'appliquer notre régression linéaire nous avons décider de regroupe toutes nos données dans un tableau ou toutes les données d'une année correspondantes sont étalés sur une ligne, pour les données qui était décrites par mois nous avons choisit de les regrouper par année en utilisant la moyenne

### Creation d'un dataframe correspondant au années et à la moyenne annuelle de "Mean Temp (°C)" 

In [2096]:
meantemp_par_an = copy(meteo)
meantemp_par_an[!, "Date"] = [Dates.year(item) for item in meantemp_par_an[!, "Date"]]
meantemp_par_an = combine(groupby(meantemp_par_an, :Date), :meantemp => mean)

,Date,meantemp_mean
,Int64,Float64
1,1999,8.27338
2,2000,6.43885
3,2001,8.01671
4,2002,7.55123
5,2003,6.43589
6,2004,6.24918
7,2005,7.46301
8,2006,8.20247
9,2007,6.89753


### Creation d'un dataframe correspondant au années et à la moyenne annuelle de "Total Snow (cm)" 

In [2097]:
total_snow_par_an = copy(meteo)
total_snow_par_an[!, "Date"] = [Dates.year(item) for item in total_snow_par_an[!, "Date"]]
total_snow_par_an = combine(groupby(total_snow_par_an, :Date), :SnowTotal => mean)

,Date,SnowTotal_mean
,Int64,Float64
1,1999,0.470461
2,2000,0.811202
3,2001,0.617808
4,2002,0.563562
5,2003,0.497808
6,2004,0.365574
7,2005,0.51726
8,2006,0.335068
9,2007,0.892877


### Creation d'un dataframe correspondant au années et à la moyenne annuelle de "Snow on Grnd (cm)" 

In [2098]:
snow_on_ground_an = copy(meteo)
snow_on_ground_an[!, "Date"] = [Dates.year(item) for item in snow_on_ground_an[!, "Date"]]
snow_on_ground_an = combine(groupby(snow_on_ground_an, :Date), :SnowGround => mean)

,Date,SnowGround_mean
,Int64,Float64
1,1999,2.08219
2,2000,2.46995
3,2001,7.70685
4,2002,1.68767
5,2003,2.56164
6,2004,2.45902
7,2005,2.71507
8,2006,2.31233
9,2007,3.65753


### Creation d'un dataframe correspondant au années et à la moyenne annuelle de "Total Rain (mm)" 

In [2099]:
rain_total_par_an = copy(meteo)
rain_total_par_an[!, "Date"] = [Dates.year(item) for item in rain_total_par_an[!, "Date"]]
rain_total_par_an = combine(groupby(rain_total_par_an, :Date), :raintotal => mean)

,Date,raintotal_mean
,Int64,Float64
1,1999,2.10712
2,2000,1.85191
3,2001,1.47096
4,2002,1.63233
5,2003,2.54767
6,2004,2.0847
7,2005,2.83452
8,2006,3.35671
9,2007,1.8211


#### Creation d'un dataframe correspondant au années et à la moyenne annuelle du prix de l'essence à québec

In [2100]:
essence_par_an_quebec = copy(essence)
essence_par_an_quebec[!, "Mois"] = [Dates.year(item) for item in essence_par_an_quebec[!, "Mois"]]
essence_par_an_quebec = combine(groupby(essence_par_an_quebec, :Mois), :Québec => mean)

,Mois,Québec_mean
,Int64,Float64
1,1997,60.5778
2,1998,55.1667
3,1999,61.4583
4,2000,71.9417
5,2001,73.95
6,2002,72.1333
7,2003,77.8
8,2004,87.0417
9,2005,97.4833


### On a décider de garder uniquements les données des années entre 1999 et 2021 afin que notre dataframe soit de la meme taille que les données de la méteo

In [2101]:
essence_par_an_quebec = essence_par_an_quebec[3:25, :]

,Mois,Québec_mean
,Int64,Float64
1,1999,61.4583
2,2000,71.9417
3,2001,73.95
4,2002,72.1333
5,2003,77.8
6,2004,87.0417
7,2005,97.4833
8,2006,102.367
9,2007,106.442


### Creation d'un dataframe correspondant au années et à la moyenne annuelle du prix de l'essence à Montréal

In [2102]:
essence_par_an_montreal = copy(essence)
essence_par_an_montreal[!, "Mois"] = [Dates.year(item) for item in essence_par_an_montreal[!, "Mois"]]
essence_par_an_montreal = combine(groupby(essence_par_an_montreal, :Mois), :Montréal => mean)

,Mois,Montréal_mean
,Int64,Float64
1,1997,61.6222
2,1998,56.2667
3,1999,63.0417
4,2000,77.1583
5,2001,73.75
6,2002,71.3583
7,2003,76.6833
8,2004,85.75
9,2005,96.35


### On a décider de se de garder selement les données des années entre 1999 et 2021 afin que notre dataframe soit de la meme taille que les données de la méteo

In [2103]:
essence_par_an_montreal = essence_par_an_montreal[3:25, :]

,Mois,Montréal_mean
,Int64,Float64
1,1999,63.0417
2,2000,77.1583
3,2001,73.75
4,2002,71.3583
5,2003,76.6833
6,2004,85.75
7,2005,96.35
8,2006,100.767
9,2007,104.333


### on regroupe les donnée de la meteo dans un seul data frame, on remarque ici que les données de l'année 2013 sans manquantes pour la meteo

In [2104]:
#On regroupe tout dans le même dataframe (je crois)

df = DataFrame(Année = meantemp_par_an[!, "Date"] ,
meantemp = meantemp_par_an[!, "meantemp_mean"],
SnowTotal = total_snow_par_an[!, "SnowTotal_mean"],
SnowGround = snow_on_ground_an[!, "SnowGround_mean"], 
raintotal = rain_total_par_an[!, "raintotal_mean"],
)


,Année,meantemp,SnowTotal,SnowGround,raintotal
,Int64,Float64,Float64,Float64,Float64
1,1999,8.27338,0.470461,2.08219,2.10712
2,2000,6.43885,0.811202,2.46995,1.85191
3,2001,8.01671,0.617808,7.70685,1.47096
4,2002,7.55123,0.563562,1.68767,1.63233
5,2003,6.43589,0.497808,2.56164,2.54767
6,2004,6.24918,0.365574,2.45902,2.0847
7,2005,7.46301,0.51726,2.71507,2.83452
8,2006,8.20247,0.335068,2.31233,3.35671
9,2007,6.89753,0.892877,3.65753,1.8211


### On remplace ici les données manquantes de 2013 par la moyenne des valeurs trouvés pour la meteo, puis on retrie les données selon l'année

In [2105]:

push!(df, [2013 mean(meantemp_par_an[!, "meantemp_mean"]) mean(total_snow_par_an[!, "SnowTotal_mean"]) mean(snow_on_ground_an[!, "SnowGround_mean"]) mean(rain_total_par_an[!, "raintotal_mean"])] )
df = sort(df)

,Année,meantemp,SnowTotal,SnowGround,raintotal
,Int64,Float64,Float64,Float64,Float64
1,1999,8.27338,0.470461,2.08219,2.10712
2,2000,6.43885,0.811202,2.46995,1.85191
3,2001,8.01671,0.617808,7.70685,1.47096
4,2002,7.55123,0.563562,1.68767,1.63233
5,2003,6.43589,0.497808,2.56164,2.54767
6,2004,6.24918,0.365574,2.45902,2.0847
7,2005,7.46301,0.51726,2.71507,2.83452
8,2006,8.20247,0.335068,2.31233,3.35671
9,2007,6.89753,0.892877,3.65753,1.8211


### ici on ajoute le pris de l'essence au québec et à Montréal à notre jeu de données.

In [2106]:

df = hcat(df, essence_par_an_quebec[!, "Québec_mean"])
df = rename(df, :"x1" => :Québec)   
df = hcat(df, essence_par_an_montreal[!, "Montréal_mean"])
df = rename(df, :"x1" => :Montréal)


,Année,meantemp,SnowTotal,SnowGround,raintotal,Québec,Montréal
,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,1999,8.27338,0.470461,2.08219,2.10712,61.4583,63.0417
2,2000,6.43885,0.811202,2.46995,1.85191,71.9417,77.1583
3,2001,8.01671,0.617808,7.70685,1.47096,73.95,73.75
4,2002,7.55123,0.563562,1.68767,1.63233,72.1333,71.3583
5,2003,6.43589,0.497808,2.56164,2.54767,77.8,76.6833
6,2004,6.24918,0.365574,2.45902,2.0847,87.0417,85.75
7,2005,7.46301,0.51726,2.71507,2.83452,97.4833,96.35
8,2006,8.20247,0.335068,2.31233,3.35671,102.367,100.767
9,2007,6.89753,0.892877,3.65753,1.8211,106.442,104.333


In [2107]:
feature_2021 = df[23:23,:]
feature_2020 = df[22:22,:]
df = df[11:22,:]

,Année,meantemp,SnowTotal,SnowGround,raintotal,Québec,Montréal
,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,2009,6.67945,0.474247,3.73973,2.19616,97.6667,97.7417
2,2010,8.46959,0.555068,1.84384,2.73918,106.458,107.842
3,2011,8.02712,0.510137,2.70685,3.01205,128.025,130.492
4,2012,8.53361,0.525683,2.01913,1.98306,134.508,136.517
5,2013,7.49198,0.573133,3.25219,2.29589,135.208,136.892
6,2014,6.86712,0.504658,3.30137,2.33167,133.042,136.925
7,2015,7.13945,0.530462,3.45205,2.45376,110.083,116.45
8,2016,8.09633,0.530238,2.10383,2.3105,101.517,108.308
9,2017,7.63394,0.646964,2.67123,2.83751,111.933,118.917


#### Pour chaque zone on crée un dataframe des données de meteo puis ceux de l'essence puis un rajoute le nombre de cerfs récoltés dans cette zone à ce jeu de donnée

In [2108]:
df_vector = [DataFrame() for _ in 1:19]

#On rajoute dans la matrice X de chaque zone, les colonnes liées aux récoltes
for i in 1:19
        first_zone = groupby(recolte, :Zone)[i]
        first_zone = combine(groupby(first_zone, :Année), :Cerfs => sum)
        n = nrow(first_zone)
        b = round(mean(groupby(recolte, :Zone)[i][!, :Cerfs]))
        for i=1999:2020
                push!(first_zone, [i b] )
        end
        first_zone = combine(groupby(first_zone, :Année), :Cerfs_sum => sum)
        first_zone = first_zone[11:22,:]#Idriss : pour enlever l'année 2020. Pour entrainer sur l'anée 2020 comprise faire : first_zone[1:22,:]
        df_vector[i] = hcat(df, first_zone[!, "Cerfs_sum_sum"])
        df_vector[i] = rename(df_vector[i], :"x1" => :Cerfs)

end

##### le jeu de données permis.csv contient plusieurs donnée manquantes pour le nombre de permis résidents premier abbatage donc on les remplace par des la moyenne de cette colones 

In [2109]:
#Remplacer les premiers abbatages manquants
replace!(permis."Résidents premier abbatage", missing => round(mean(skipmissing(permis[!, "Résidents premier abbatage"]))))
permis_copy = copy(permis)
permis = permis[12:23, :] 


,Année,Résidents,Résidents premier abbatage,Non-résidents,Total
,Int64,Int64,Int64?,Int64,Int64
1,2009,152388,8826,1231,162445
2,2010,145491,8635,1108,155234
3,2011,140968,8369,1114,150451
4,2012,143470,8795,1071,153336
5,2013,144980,8916,1049,154945
6,2014,143829,8883,1028,153740
7,2015,140789,9286,1020,151095
8,2016,139384,9351,943,149678
9,2017,137224,9723,913,147860


#### on ajoute a notre chacun de nos dataframe correspondant à chaque zone le nombre de permis attribués annuellement.

In [2110]:

#On rajoute dans la matrice X de chaque zone, les colonnes liées au permis
for i in 1:19
    df_vector[i] = hcat(df_vector[i], permis[!, "Résidents premier abbatage"])
    df_vector[i] = rename(df_vector[i], :"x1" => :"Résidents 1er")
    df_vector[i] = hcat(df_vector[i], permis[!, "Non-résidents"])
    df_vector[i] = rename(df_vector[i], :"x1" => :"Non-résidents")
    df_vector[i] = hcat(df_vector[i], permis[!, "Résidents"])
    df_vector[i] = rename(df_vector[i], :"x1" => :"Résidents")
end


# 3. Feature Design

Voici les règles que nous avons suivis pour le choix des features : 

D’après la matrice de corrélation vue dans l’analyse exploratoire, certaines features sont fortement corrélés, ainsi la première règle à été de ne jamais tester un modèle contenant 2 ou plus features corrélées.

Nous avons considéré la feature “Engins” qui serait un OneHot qui indiquerait quels engins ont été utilisés. Cependant, nous entraînons un modèle par zone, or les engins varient très rarement par zone, ce qui implique que cette feature aurait une variance proche de 0 (pour chaque zone), ainsi nous avons préféré la retirer.

Nous avons séparé certaines features en plusieurs, car les garder en une seule était une perte d’informations. Par exemple, nous avons séparé la feature “Essence” en 2, “Québec” et “Montréal” afin d’utiliser “Québec” pour les zones se trouvant au Québec, et “Montréal” pour les zones se trouvant proche de Montréal. De même pour la feature “Permis”, que nous avons séparé en 3, “Résidents”, “Non-Résidents” et “1er abattage". L’intuition derrière ce choix est qu’il serait intéressant par exemple d’utiliser “Non-Résident” et “Essence” dans le même modèle, car les non résidents doivent parcourir une plus grande distance pour aller chasser. Si on avait seulement une feature “Permis”, l’information du nombre de personnes devant parcourir de grandes distances serait perdue.


Comme nous allons le présenter dans la partie suivante (4. Modèles), nous allons utiliser un modèle de regréssion polynomiale pénalisée. Il nous faut donc un moyen d'augmenter les degrés de chaque feature, c'est le rôle de la fonction "get_polynomial_feature". Elle prend en paramètre une colonne de datarframe et rend en sortie des nouvelles colonnes, une pour chaque degré.

In [2111]:
#Cette fonction permet de rajouter des colonnes pour la feature en argument, pour chaque degré
# Par exemple pour modifier et que ça transforme une feature au degré 5, il faut uncomment ce que tout ce qui est commenté 
function get_polynomial_feature(array,feature_name)
    a = "2"
    b = "3"
    c = "4"
    d = "5"
    e = "6"

    arr = copy(array)
    arr2 = arr.^2
    arr3 = arr.^3
    arr4 = arr.^4
    arr5 = arr.^5
    arr6 = arr.^6
    
    # df = DataFrame([arr2,arr3],[feature_name * a,feature_name * b])
    df = DataFrame([arr2,arr3],[feature_name * a,feature_name *b])
    return df
end

get_polynomial_feature (generic function with 1 method)

Cette cellule permet de sélectionner les features que l'on souhaite garder dans notre modèle. Comme expliqué précédemment, le dataframe "df_vector" contient chacune de nos matrices de structure X. Ainsi on sélectionne celles qui nous intéressent. Pour notre modèle final, nous avons conservé l'année, la quantité de neige sur le sol ainsi que la quantité de pluie car il nous donne notre meilleur score.

In [2112]:
for i in 1:19
    df_vector[i] = select!(df_vector[i], [:Année,:SnowGround,:raintotal,:Cerfs])
end

Ensuite, on transforme en polynôme les cellules de notre choix. Il n'est pas nécessaire de transformer en polynôme chacune des cellules choisies

In [2113]:
# On ajoute à chaque matrice X (une matrice X par zone car un modèle par zone) toutes les features suivantes en polynome
for i in 1:19
    # df_vector[i] = hcat(df_vector[i],get_polynomial_feature(df_vector[i].meantemp,"meanTemp"))
    df_vector[i] = hcat(df_vector[i],get_polynomial_feature(df_vector[i].Année,"Année"))
    df_vector[i] = hcat(df_vector[i],get_polynomial_feature(df_vector[i].SnowGround,"SnowGround"))
    df_vector[i] = hcat(df_vector[i],get_polynomial_feature(df_vector[i].raintotal,"rainTotal"))
    # df_vector[i] = hcat(df_vector[i],get_polynomial_feature(df_vector[i].Québec,"Québec"))
    # df_vector[i] = hcat(df_vector[i],get_polynomial_feature(df_vector[i].Montréal,"Montréal"))
    # df_vector[i] = hcat(df_vector[i],get_polynomial_feature(df_vector[i][!,:"Résidents 1er"],"Résidents 1er"))
    # df_vector[i] = hcat(df_vector[i],get_polynomial_feature(df_vector[i].Résidents,"Résidents"))
    # df_vector[i] = hcat(df_vector[i],get_polynomial_feature(df_vector[i][!,:"Non-résidents"],"Non-résidents"))
end

In [2114]:
df_vector[17] #On check que tout s'est bien passé

,Année,SnowGround,raintotal,Cerfs,Année2,Année3,SnowGround2,SnowGround3
,Int64,Float64,Float64,Int64,Int64,Int64,Float64,Float64
1,2009,3.73973,2.19616,61,4036081,8108486729,13.9856,52.3021
2,2010,1.84384,2.73918,67,4040100,8120601000,3.39973,6.26854
3,2011,2.70685,3.01205,76,4044121,8132727331,7.32703,19.8332
4,2012,2.01913,1.98306,111,4048144,8144865728,4.07687,8.23171
5,2013,3.25219,2.29589,108,4052169,8157016197,10.5767,34.3976
6,2014,3.30137,2.33167,96,4056196,8169178744,10.899,35.9818
7,2015,3.45205,2.45376,26,4060225,8181353375,11.9167,41.137
8,2016,2.10383,2.3105,102,4064256,8193540096,4.42608,9.3117
9,2017,2.67123,2.83751,114,4068289,8205738913,7.13549,19.0605


Ensuite, il faut créer feature_2021, qui regroupe les observations de 2021 pour les features choisies, en gros il s'agit de X_test. Il faut bien évidemment garder le même format que pour notre échantillon d'entrainement ainsi dans cette cellule on applique la même chose que ce qui a été fait pour créer nos matrices de structure. 

De plus, comme nous le verrons dans la partie Optimisation, nous avons utilisé l'année 2020 comme validation set afin de pouvoir tester nos modèles sans devoir soumettre sur kaggle à chaque fois. On fait donc la même chose pour 2020 en créeant feature_2020

In [2115]:
#On créer le vecteur x pour l'année 2021, qu'on utilisera comme entrée du modèle pour faire les prédictions

feature_2021 = convert(DataFrame, feature_2021)
feature_2021 = hcat(feature_2021, permis_copy[!, "Résidents premier abbatage"][23:23])
feature_2021 = rename(feature_2021, :"x1" => :"Résidents 1er")
feature_2021 = hcat(feature_2021, permis_copy[!, "Non-résidents"][23:23])
feature_2021 = rename(feature_2021, :"x1" => :"Non-résidents")
feature_2021 = hcat(feature_2021, permis_copy[!, "Résidents"][23:23])
feature_2021 = rename(feature_2021, :"x1" => :"Résidents")

#Même chose pour l'année 2020

feature_2020 = convert(DataFrame, feature_2020)
feature_2020 = hcat(feature_2020, permis_copy[!, "Résidents premier abbatage"][22:22])
feature_2020 = rename(feature_2020, :"x1" => :"Résidents 1er")
feature_2020 = hcat(feature_2020, permis_copy[!, "Non-résidents"][22:22])
feature_2020 = rename(feature_2020, :"x1" => :"Non-résidents")
feature_2020 = hcat(feature_2020, permis_copy[!, "Résidents"][22:22])
feature_2020 = rename(feature_2020, :"x1" => :"Résidents")


,Année,meantemp,SnowTotal,SnowGround,raintotal,Québec,Montréal,Résidents 1er
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Int64?
1,2020,8.2331,0.522951,3.56284,2.37541,103.567,106.25,9397


Ensuite on transforme nos set de validation et de test en polynôme, toujours pour qu'ils aient le même format que notre matrice de structure

In [2116]:
#On Transforme chaque feature en polynome, comme ce qui a été fait pour l'ensemble d'entrainement. 

feature_2020 = select!(feature_2020, [:Année,:SnowGround,:raintotal])
feature_2021 = select!(feature_2021, [:Année,:SnowGround,:raintotal])

# On fait ça pour le Test set : 2021
# feature_2021 = hcat(feature_2021,get_polynomial_feature(feature_2021.meantemp,"meanTemp"))
feature_2021 = hcat(feature_2021,get_polynomial_feature(feature_2021.Année,"Année"))
feature_2021 = hcat(feature_2021,get_polynomial_feature(feature_2021.SnowGround,"SnowGround"))
feature_2021 = hcat(feature_2021,get_polynomial_feature(feature_2021.raintotal,"rainTotal"))
# feature_2021 = hcat(feature_2021,get_polynomial_feature(feature_2021.Québec,"Québec"))
# feature_2021 = hcat(feature_2021,get_polynomial_feature(feature_2021.Montréal,"Montréal"))
# feature_2021 = hcat(feature_2021,get_polynomial_feature(feature_2021[!,:"Résidents 1er"],"Résidents 1er"))
# feature_2021 = hcat(feature_2021,get_polynomial_feature(feature_2021.Résidents,"Résidents"))
# feature_2021 = hcat(feature_2021,get_polynomial_feature(feature_2021[!,:"Non-résidents"],"Non-résidents"))

# On fait ça pour le Validation set : 2020
# feature_2020 = hcat(feature_2020,get_polynomial_feature(feature_2020.meantemp,"meanTemp"))
feature_2020 = hcat(feature_2020,get_polynomial_feature(feature_2020.Année,"Année"))
feature_2020 = hcat(feature_2020,get_polynomial_feature(feature_2020.SnowGround,"SnowGround"))
feature_2020 = hcat(feature_2020,get_polynomial_feature(feature_2020.raintotal,"rainTotal"))
# feature_2020 = hcat(feature_2020,get_polynomial_feature(feature_2020.Québec,"Québec"))
# feature_2020 = hcat(feature_2020,get_polynomial_feature(feature_2020.Montréal,"Montréal"))
# feature_2020 = hcat(feature_2020,get_polynomial_feature(feature_2020[!,:"Résidents 1er"],"Résidents 1er"))
# feature_2020 = hcat(feature_2020,get_polynomial_feature(feature_2020.Résidents,"Résidents"))
# feature_2020 = hcat(feature_2020,get_polynomial_feature(feature_2020[!,:"Non-résidents"],"Non-résidents"))


,Année,SnowGround,raintotal,Année2,Année3,SnowGround2,SnowGround3,rainTotal2
,Int64,Float64,Float64,Int64,Int64,Float64,Float64,Float64
1,2020,3.56284,2.37541,4080400,8242408000,12.6938,45.2261,5.64257


In [2117]:
feature_2021

,Année,SnowGround,raintotal,Année2,Année3,SnowGround2,SnowGround3,rainTotal2
,Int64,Float64,Float64,Int64,Int64,Float64,Float64,Float64
1,2021,5.69863,1.71205,4084441,8254655261,32.4744,185.06,2.93113


# 4. Modèle

Dans le cadre de notre recherche de modèle, nous avons tout d'abord remarquer qu'en fonction de la zone prise en compte, la distributions du nombre des cerfs récolté par années est différentes en fonction des zones. Cette information nous a donc pouser a envisager des modèles différent en fonction de chaque zones.

Dans un premier temps, nous avons essayer des modelèle de regression multiple avec comme variables explicatives chacunes des variables qui nous ont été fournis. Ces modèle avait l'air de bien fonctionner et ont donc fait l'affaire de nos première soumission sur kaggle . 

Cependant après visualisation des données, et notamment les variables explicatves les plus importantes pour la régression, nous avons remarquée que les plupart des relations n'etaient pas linéaires, mais avais plutot l'air polynomiale. Aisni, nous avons donc décidée de mettre à jour notre méthodolgie de determination de modèle. Nous somme don passées à la régression multiple polynomiales. 

La sélection des variables explicatives, à passer en polynôme serra expliquer plus tard dans une section ultérieure. Néanmoins, et comme cité précédemment nous n'avons as passer toutes les variables à des degrés supérieurs : on a réalisé une sélection de ces dernières. L'on obtient donc des modèles avec un mélange de régression linéaire polynomiale et simple.

De plus, lors de notre analyse exploratoire, nous avons pu remarquer que certaines variables explicatives sont fortement corrélé et peuvent induires des problèmes de multicolinéarité. Ce phénomène peut créer des instabilitées numériques lors du calcul de l'inverse de certaines matrice et donc induire à l'obtention de coefficient avec de trop grande valeurs.

De même, nous avons aussi visualisé, lors de phases de test, avec un set de validation que notre modèle réalisé du surapprentissage sur nos données d'entrainement et avait ainsi du mal à généraliser ces prédictions sur le set de test.

Donc, pour ce faire, nous avons donc décidé de passer à de la régression pénaliser. Les résultats obtenus avec ces derniers modèles de régression étaient ainsi les meilleurs. Et nous avons donc décidé de les garder pour nos derniers modèles



Dans cette cellule nous réalisons la création du dataframe qui va comporter les différent coefficient de régression en fonction des zones

In [2119]:
#On créer le dataframe qui comportera tout nos coefficients, 
models = DataFrame(Zone = Int64[])
for i in 1:10 #Changer le nombre si on a modifié le degré de la reg polynomiale
    B_str = "B"
    i_str = string(i)
    b = DataFrame(B0 = Float64[])
    b = rename(b, :B0 => B_str * i_str)
    models = hcat(models,b)
end
models

,Zone,B1,B2,B3,B4,B5,B6,B7,B8,B9
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64


En ce qui concerne la cellule suivante, nous réalisons la fonction qui nous permet d'entrainer tous nos modèles sur nos données d'entrainement

In [2120]:
#On train le modèle puis on met tout les coefficients dans le dataframe "models"
λ = 1e6 # on défini lambda

   # on fait la régression pour trouver les coefs.

i=1
num = [1,2,3,4,5,6,7,8,9,10,11,12,13,15,20,21,26,27,28]
for zone in df_vector  # Pour chaque zone de chasse,
    
    y = zone.Cerfs

    n = length(y)
    
    z = copy(zone)
    X = select!(z, Not([:Cerfs]))

    X = hcat(ones(n), Matrix(X[:,:]))

    B = (X'X + λ*I)\X'y  # Estimation des coefficients de régression
    
    push!(models, vcat(num[i],B))
    i=i+1
end


Cette cellule sert à visualiser tous les coefficients de régressions obtenus après avoir entrainé nos modèles. veuillez noter que les chiffres sont incrémenté de 1 à 19. Cependant ce chiffre ne correspond pas au numéro réel de la zone mais à un numéro attribué à cette dernière pour faciliter la réalisation des fonctions.

In [2121]:
models

,Zone,B1,B2,B3,B4,B5,B6
,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,-1.94503e-8,1.17377e-6,-0.000645863,-0.000218612,0.0286936,-1.40837e-5
2,2,-9.81714e-9,-3.08459e-6,-0.00051938,-0.000278729,0.00706246,-3.39387e-6
3,3,-4.5593e-8,-4.22712e-5,-0.00102633,-0.000377487,-0.0234896,1.19138e-5
4,4,-7.54455e-8,-8.17872e-5,-0.0013695,-0.000341103,-0.0626556,3.15964e-5
5,5,-1.55359e-8,-2.48828e-5,-0.000364763,1.62184e-5,-0.0290789,1.46958e-5
6,6,-1.97173e-8,-1.39787e-5,-0.000434295,-0.00116149,-0.00149733,1.38334e-6
7,7,-4.09627e-8,-5.31566e-5,-0.0013281,-0.000197277,-0.0516658,2.60832e-5
8,8,-1.89093e-8,-4.03905e-5,-0.000246282,6.21683e-5,-0.0557834,2.81162e-5
9,9,-4.50947e-9,-2.69918e-6,-0.000415731,-0.000321256,0.000656588,-1.60887e-7


# 5. Prédictions

Cette partie regroupe tout le code qui permet de faire des prédictions, nous nous sommes fortement inspités de l'extrait de code qui était déjà fournie.

In [2122]:
models = models[in.(models.Zone, Ref(test.Zone)), :];  # On récupère seulement les modèles pour les zones à prévoir

In [2123]:
# dataframe avec seulement les coefficients, puis on transforme ça en matrice pour pouvoir lu'tiliser pour faire les prédictions
beta = select!(models, Not([:Zone]))
beta = Matrix(beta[:,:])
#On passe x en matrice et on ajoute la colonne des 1 ==> c'est notre X_test
x_2021 = Matrix(feature_2021[:,:])
x_2021 = hcat(1,x_2021)

#On passe x en matrice et on ajoute la colonne des 1 ==> c'est notre X_validation
x_2020 = Matrix(feature_2020[:,:])
x_2020 = hcat(1,x_2020)

1×10 Matrix{Float64}:
 1.0  2020.0  3.56284  2.37541  …  12.6938  45.2261  5.64257  13.4034

In [2124]:
# Prédictions sur le validation set
ŷ_validation = [Int64(ceil(disallowmissing(x_2020 * beta[i,:])[1] )) for i in 1:size(test,1)];

In [2125]:
#On met nos prédictions de validation dans un csv

prediction = DataFrame(Zone = test.Zone, Cerfs = ŷ_validation)
CSV.write("validation_pred.csv", prediction)

"validation_pred.csv"

In [2126]:
# On fait nos prédictions, pour chaque zone i, on prédit en faisant x*Bi avec Bi ke vecteur de coeffs pour la zone i
ŷ_pred = [Int64(ceil(disallowmissing(x_2021 * beta[i,:])[1] )) for i in 1:size(test,1)];

In [2127]:
#On met nos prédictions dans un csv

prediction = DataFrame(Zone = test.Zone, Cerfs = ŷ_pred)
CSV.write("pred.csv", prediction)

"pred.csv"

In [2128]:
zones = groupby(recolte, :Zone)  # On groupe les données par zone

new_df = DataFrame(Année = Int64[], Zone = Int64[], Cerfs= Int64[])  # On initialise un DataFrame vide

for zone in zones  # Pour chaque zone de chasse,
    
    # pour chaque année,
    # on additionne le nombre de cerfs récoltés selon les différents engins de chasse
    df = combine(groupby(zone, :Année), :Cerfs => sum => :Cerfs)  
    
    df2 = DataFrame(Année = df.Année, 
                    Zone = fill(unique(zone.Zone)[1], size(df, 1)),
                    Cerfs = df.Cerfs) 
    
    append!(new_df, df2)  # On ajoute l'information au DataFrame préinitialisé 
end

sort!(new_df, :Année)
y_2020 = last(new_df, 18)  

,Année,Zone,Cerfs
,Int64,Int64,Int64
1,2020,1,318
2,2020,2,552
3,2020,3,1558
4,2020,4,3146
5,2020,5,2239
6,2020,6,4307
7,2020,7,4055
8,2020,8,4464
9,2020,9,1089


# 6. Optimisation 

Dans le but d'optenir de meilleurs performances, nous avons utilisé plusieus méthodes que nous allons expliquer dans cette partie.

La première méthode que nous avons utilisé a été précedemment mentionné dans la partie feature design. Elle consiste à ne pas tester dans un modèle 2 features fortements corrérlées (information que nous avons obtenu grâce à la matrice de corrélation), car nous savons que le vecteur Beta inverser la matrice de structure ce qui fait qu'il existe beaucoup de valeurs possible de Beta. En somme, beta aura une trop grande variance si certaines features sont corrélées, ce qui pourrait fortement impacté la capacité de généralisation de notre modèle. En effet, si la valeur de Beta est trop grande, alors la valeur qui minimise l'erreur de validation peut être très différente de celle qui minimise l'erreur de test. La terme de pénalisation lambda devrait normalement résoudre ce problème, cependant nous avons quand même préférer ne pas prendre le risque.

La deuxième méthode d'optimisation que nous avons utilisé a été de ne pas transformer en polynôme toutes les features du modèle. La manière dont nous l'avons programmé permet toute liberté sur le choix des features qui seront transformés en polynôme. Cette décision permet dans certains cas de limiter le surentraînement tout augmantant le nombre de combinaisons de features possibles.

Et pour finir, la méthode principale que nous avons utilisé est le hyperparameter tuning, qui correspond à l'acte de tester différentes valeurs pour notre hyperparamètre lambda. Pour chaque modèle, chaque combinaisons de feature, nous testions à chaque fois plusieurs valeurs pour lambda. Cela est très utile, car il arrive souvant qu'un modèle soit mauvais pour une certaine valeur de lambda, mais très bon pour une autre. Dans ce genre de cas par exemple, nous aurions simplement abandonné le modèle en pensant qu'il avait de mauvaises performances alors que ce n'est pas le cas. C'est d'ailleurs ce qu'il est arrivé pour notre modèle finale, qui utilise une valeur pour lambde de 10 à la puissance 6. Sans avoir effectué d'hyperparameter tuning, nous n'aurions jamais réalisé que le modèle était bon.

# Score validation

Nous utilisons l'année 2021 pour tester nos modèles, car il s'agit de l'année dont les nombres de cerfs récoltés par zones sont les plus similaires à l'année 2021. Pour calculer le score, nous utilisons le RMSE afin de reproduire avec le plus de précision les scores sur kaggle. Cela nous permet d'être sûr qu'un modèle obtiendra un bon score sur kaggle s'il a un bon score de validation. A titre d'exemple, notre modèle final nous donne un score de 527 sur le leaderbord, et 526,66 en score de validation. On peut dire que c'est très précis.

In [2129]:
y_2020_Mat = Matrix(y_2020.Cerfs[:,:])
error = y_2020_Mat - ŷ_validation
mean(abs.(error))

526.6666666666666

# Amélioration

Parmis les améliorations possibles qu'on pourrait faire:

*  La normalisations PCA pourrait aider à ameliorer notre score, cependant cette amélioration ne sera pas très significative puisque nous soupconons que le plus gros problème dans notre prédictions est notre modèle.

* Il aurait été possible d'élargir le jeu de données des permis en assignant une proportions du nombre de permis à chaque zone, en s'appuyant sur le nombre de cerfs capturés dans chacune des zones.

* Il aurait été aussi possible d'utiliser une régression bayésienne pour prédira le nombre de permis en 2021, au lieu de prendre celui de l'année précedente

* Le fait que l'on possède pas une loi à priori informative nous a découragé d'utiliser une régression bayésienne cependant on réalise qu'il aurait été possible de s'en servir d'une loi non informative.